In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
season_code = "25_26"
current_dir = Path.cwd()  
path_folder = current_dir.parent.parent.parent.parent / "csv" / f"csv{season_code}" / "players" / "centiles"

In [3]:
def add_derived_columns(df):
    df = df.copy()

    df["Ball Losses"] = (
        df.get("Dispossessed", 0) + df.get("Miscontrols", 0)
    )
    
    df["Progressive Actions (Total)"] = (
        df.get("Progressive Passes", 0)
        + df.get("Progressive Runs", 0)
        + df.get("Progressive Carries", 0)
        + df.get("Progressive Passes Received", 0)
    )
    
    df["Actions created"] = (
        df.get("Shot-Creating Actions (SCA)", 0)
        + df.get("Goal-Creating Actions (GCA)", 0)
    )
    
    df["Actions in the Penalty Area"] = (
        df.get("Crosses into Penalty Area", 0)
        + df.get("Passes into Penalty Area", 0)
        + df.get("Touches in Attacking Penalty Area", 0)
        + df.get("Carries into Penalty Area", 0)
    )
    
    df["Total Duels Won"] = (
        df.get("Tackles Won", 0) + df.get("Dribblers Tackled", 0)
    )
    
    df["Efficiency"] = (
        df.get("Goals", 0)
        - df.get("Expected Goals (xG)", 0)
    )
    return df

In [4]:
def add_parameters(df):
    df = df.copy()
    
    df["% Efficiency"] = (
        (df.get("Goals", 0)
        - df.get("xG", 0))/df.get("xG", 1)
    )
    
    df["% Aerial Duels"] = (
        100 * df.get("Aerials Won", 0) / df.get("Total Aerials", 1)
    )
    
    df["% Passes (Total)"] = (
        100 * df.get("Passes Completed (Total)", 0) / df.get("Passes Attempted (Total)", 1)
    )
    
    df["% Passes (Short)"] = (
        100 * df.get("Passes Completed (Short)", 0) / df.get("Passes Attempted (Short)", 1)
    )
    
    df["% Passes (Medium)"] = (
        100 * df.get("Passes Completed (Medium)", 0) / df.get("Passes Attempted (Medium)", 1)
    )
    
    df["% Passes (Long)"] = (
        100 * df.get("Passes Completed (Long)", 0) / df.get("Passes Attempted (Long)", 1)
    )
    
    df["% Tackles/Duels"] = (
        100 * ( df.get("Tackles Won", 0) + df.get("Dribblers Tackled", 0) ) / (df.get("Tackles", 0) + df.get("Dribbles Challenged", 0))
    )

    
    df["% Take-Ons"] = (
        100 * df.get("Successful Take-Ons", 0) / df.get("Take-Ons Attempted", 1)
    )
    
    return df

In [5]:
def aggregated_data(data):
    df = add_derived_columns(data)
    static_cols = ['Nationality', 'Team', 'League', 'Position', 'General Position', 'Age']
    
    
    numeric_cols = [col for col in df.columns if col not in static_cols and pd.api.types.is_numeric_dtype(df[col])]

    df['Matches'] = 1  
    df_aggregated = df.groupby(['Player', 'Age'], as_index=False).agg({
        **{col: 'first' for col in static_cols if col in df.columns},
        **{col: 'sum' for col in numeric_cols},
        'Matches': 'count'
    })


    return df_aggregated

In [6]:
def adjusted_data(df_aggregated):
    df_aggregated = df_aggregated.copy()
    
    minutes = df_aggregated['Minutes'].replace(0, np.nan)  

    exclude_cols = ['Nationality', 'Team', 'League', 'Position', 'General Position', 'Age', 'Minutes', 'Matches']
    
    per90_cols = [col for col in df_aggregated.columns if col not in exclude_cols and pd.api.types.is_numeric_dtype(df_aggregated[col])]
    
    for col in per90_cols:
        df_aggregated[col] = (df_aggregated[col] * 90 / minutes).round(2)
    
    df_aggregated['Minutes'] = minutes.fillna(0).round(0).astype(int)
    df_adjusted = add_parameters(df_aggregated).round(2)
    df_adjusted = df_adjusted.fillna(0)

    return df_adjusted


In [7]:
def centiles_data(df_adjusted):
    df_adjusted = df_adjusted.copy()
    
    static_cols = ['Nationality', 'Team', 'League', 'Position', 'General Position', 'Age', 'Minutes', 'Matches']
    
    stat_cols = [col for col in df_adjusted.columns if col not in static_cols and pd.api.types.is_numeric_dtype(df_adjusted[col])]
    
    negative_stats = [
        'Yellow Cards', 'Red Cards', 'Second Yellow Cards',
        'Fouls Committed', 'Offsides', 'Miscontrols', 'Dispossessed',
        'Errors', 'Own Goals', 'Penalties Kicks Conceded', 'Challenges Lost',
        'Ball Losses', 'Aerials Lost'
    ]
    
    for col in negative_stats:
        if col in stat_cols:
            df_adjusted[col] = - df_adjusted[col]
    
    df_centiles = df_adjusted.copy()
    df_centiles[stat_cols] = df_adjusted.groupby('General Position')[stat_cols].transform(lambda x: x.rank(pct=True) * 100)
    
    df_centiles[stat_cols] = (df_centiles[stat_cols].replace([np.inf, -np.inf], 0).fillna(0).astype(int))
    
    return df_centiles

In [9]:
os.makedirs(path_folder, exist_ok=True)

for filename in os.listdir(path_folder):
    if filename.endswith("data_players.csv"):
        path_file = os.path.join(path_folder, filename)
        
        path_end_aggregated_data = path_folder / "data_players_aggregated.csv"
        path_end_adjusted_data   = path_folder / "data_players_adjusted.csv"
        path_end_centiles        = path_folder / "data_players_centiles.csv"

        data = pd.read_csv(path_file)
        df_aggregated = aggregated_data(data)
        df_adjusted = adjusted_data(df_aggregated)
        df_centiles = centiles_data(df_adjusted)

        df_aggregated.to_csv(path_end_aggregated_data, index=False)
        df_adjusted.to_csv(path_end_adjusted_data, index=False)
        df_centiles.to_csv(path_end_centiles, index=False)

        print(f"Fichier traité et sauvegardé : {filename}")

Fichier traité et sauvegardé : data_players.csv
